In [ ]:
library('forecast')
library('tseries')
library('CADFtest')

In [4]:
# metrics
MAPE = function(data){
    data = data[data$real != 0,]
    return (mean(abs((data$predict - data$real) / data$real)))
}

direction_accuracy = function(data){
    return (mean(ifelse(((data$real - data$prev_real) * (data$predict - data$prev_real)) >= 0, T, F)))
}

In [3]:
# load data
data = read.csv("../data/application_count_from_1960_to_2015_for_LSTM.csv")
data$date = as.Date(data$date , "%Y-%m-%d")
train = data[(data$date >= '1985-01-01') & (data$date < '2006-01-01'),]

In [12]:
head(train)

,label,date,count
301,1,1985-01-31,0
302,1,1985-02-28,0
303,1,1985-03-31,0
304,1,1985-04-30,0
305,1,1985-05-31,0
306,1,1985-06-30,0


In [ ]:
# predect

predicts = c()

h = 12 # forecast point (month)
for (cluster in split(train, train$label)){
    label = cluster$label[1]
          
    model = auto.arima(
        cluster$count,
        ic="aic",
        trace=T,
        stepwise=T,
        approximation=F,
        allowdrift=F,
        start.p=0,
        start.q=0,
        start.P=0,
        start.Q=0,
    )

    pred_ts = forecast(model, h = h)
    
    if (h == 3){
        real = data[(data$date >= '2006-01-01') & (data$date < '2006-04-01') & (data$label == label),]$count
        prev_real = data[(data$date >= '2005-10-01') & (data$date < '2006-01-01') & (data$label == label),]$count
    }
    else if (h == 12){
        real = data[(data$date >= '2006-01-01') & (data$date < '2007-01-01') & (data$label == label),]$count
        prev_real = data[(data$date >= '2005-10-01') & (data$date < '2006-01-01') & (data$label == label),]$count
    }
    predicts = rbind(
        predicts,
        c(label=label, predict=rev(pred_ts$mean)[1], real=rev(real)[1], prev_real=rev(prev_real)[1])
    )

    plot(
        pred_ts,
        shadecols=c("yellow", "orange"),
        fcol=1,
        xlim=c(220, 300),
        lwd=2
    )
    tsdiag(model)
}

In [7]:
# make dataframe of predicted values
predicts = as.data.frame(predicts)

In [8]:
print(paste("MAPE:", MAPE(predicts)))
print(paste("Direction Accuracy:", direction_accuracy(predicts)))

[1] "MAPE: 0.715466299776888"
[1] "Direction Accuracy: 0.82051282051282"
